In [ ]:
from tensorflow.keras.utils import img_to_array
from keras.preprocessing import image
import cv2
import numpy as np
from keras.models import load_model
import keras.backend as K

def f1_score(y_true, y_pred):
    y_pred_classes = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred_classes, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred_classes, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred_classes), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)




face_classifier = cv2.CascadeClassifier(r"C:\Users\nourm\OneDrive\Desktop\PROJECTS\UNITY MODEL PROJECT\CodePython\src\haarcascade_frontalface_default.xml")

# Load model with custom metric
classifier = load_model(
    r"C:\Users\nourm\OneDrive\Desktop\PROJECTS\UNITY MODEL PROJECT\CodePython\models\Emotion_model_27.h5",
    custom_objects={'f1_score': f1_score}
)

class_labels = ['angry','disgust','fear','happy','kissing','neutral','sad','sleepy','surprise']

cap = cv2.VideoCapture(0)

while True:
    # Grab a single frame of video
    ret, frame = cap.read()
    labels = []
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_classifier.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
        roi_gray = gray[y:y + h, x:x + w]
        roi_gray = cv2.resize(roi_gray, (48, 48), interpolation=cv2.INTER_AREA)

        if np.sum([roi_gray]) != 0:
            roi = roi_gray.astype('float') / 255.0
            roi = img_to_array(roi)
            roi = np.expand_dims(roi, axis=0)

            # make a prediction on the ROI, then lookup the class
            preds = classifier.predict(roi)[0]
            label = class_labels[preds.argmax()]
            label_position = (x, y)
            cv2.putText(frame, label, label_position, cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)
        else:
            cv2.putText(frame, 'No Face Found', (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 3)

    cv2.imshow('Emotion Detector', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


1/1 [==============================] - 0s 89ms/step


In [9]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
import keras.backend as K

def f1_score(y_true, y_pred):
    y_pred_classes = K.round(y_pred)
    tp = K.sum(K.cast(y_true * y_pred_classes, 'float'), axis=0)
    fp = K.sum(K.cast((1 - y_true) * y_pred_classes, 'float'), axis=0)
    fn = K.sum(K.cast(y_true * (1 - y_pred_classes), 'float'), axis=0)

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())
    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return K.mean(f1)

# Load the model
model = load_model(r"C:\Users\nourm\OneDrive\Desktop\PROJECTS\UNITY MODEL PROJECT\Complevit_FER\FER_model.h5",custom_objects={'f1_score': f1_score})

test_data_path = r"C:\Users\nourm\OneDrive\Desktop\PROJECTS\UNITY MODEL PROJECT\FER-2013 (USED)-OLD AND REMOVED - Copy\test"

# STEP 1: Load the data using ImageDataGenerator
test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize images

input_height, input_width = model.input_shape[1:3]

test_data = test_datagen.flow_from_directory(
    test_data_path,
    target_size=(input_height, input_width),
    batch_size=32,
    class_mode='categorical',
    color_mode='grayscale',  
    shuffle=False
)

print(f"Found {test_data.samples} test images belonging to {test_data.num_classes} classes")

results = model.evaluate(test_data)
print(f"Test Loss: {results[0]:.4f}")
print(f"Test Accuracy: {results[1]:.4f}")

# h=history.history
# h.keys

# import matplotlib as plt
# plt.plot(h["accuracy"],c="yellow")
# plt.show

Found 7628 images belonging to 9 classes.
Found 7628 test images belonging to 9 classes
239/239 [==============================] - 11s 40ms/step - loss: 1.2774 - accuracy: 0.6543 - precision: 0.7374 - recall: 0.5788 - f1_score: 0.0809
Test Loss: 1.2774
Test Accuracy: 0.6543
